In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os
from pathlib import Path
import xgboost as xgb
import lightgbm as lgbm
import catboost
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score
from IPython.display import display
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, OrdinalEncoder
import optuna
from sklearn.preprocessing import StandardScaler
from scipy.linalg import norm

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner as kt

from category_encoders import LeaveOneOutEncoder

from IPython.display import display
from tqdm.notebook import tqdm

In [47]:
from warnings import filterwarnings
filterwarnings("ignore")

# Loading Data

In [2]:
BASE_PATH = Path("/kaggle/input/playground-series-s3e4/")

train = pd.read_csv(BASE_PATH / "train.csv")
test = pd.read_csv(BASE_PATH / "test.csv")

print(f"Training dataset has {len(train)} rows with " \
                    f"{sum(train.Class) / len(train) * 100 :.2}% fraud rows.")

Training dataset has 219129 rows with 0.21% fraud rows.


# Preprocessing

In [3]:
train["Hour"] = ((train.Time // 3600) % 24).astype("int")
test["Hour"] = ((test.Time // 3600) % 24).astype("int")

In [4]:
def time_of_day(hour):
    if hour >=0 and hour <= 6:
        return "night"
    elif hour <= 12:
        return "morning"
    elif hour <= 18:
        return "afternoon"
    else:
        return "evening"

In [27]:
train["Hour"] = ((train.Time // 3600) % 24).astype("category")
test["Hour"] = ((test.Time // 3600) % 24).astype("category")

In [28]:
train["TimeOfDay"] = train.Hour.map(time_of_day)
test["TimeOfDay"] = test.Hour.map(time_of_day)

In [29]:
train["Day"] = (((train['Time'] // (24 * 3600)) % 7) + 1).astype("category")
test["Day"] = (((test["Time"] // (24 * 3600)) % 7) + 1).astype("category")

In [30]:
train.head()

,id,Time,V1,V2,V3,V4,V5,V6,V7,V8,...,V24,V25,V26,V27,V28,Amount,Class,Hour,TimeOfDay,Day
0,0,0.0,2.074329,-0.129425,-1.137418,0.412846,-0.192638,-1.210144,0.110697,-0.263477,...,-0.110835,-0.291459,0.207733,-0.076576,-0.059577,1.98,0,0.0,night,1.0
1,1,0.0,1.998827,-1.250891,-0.520969,-0.894539,-1.122528,-0.270866,-1.029289,0.050198,...,-0.461928,-0.465491,-0.464655,-0.009413,-0.038238,84.00,0,0.0,night,1.0
2,2,0.0,0.091535,1.004517,-0.223445,-0.435249,0.667548,-0.988351,0.948146,-0.084789,...,0.951233,-0.506919,0.085046,0.224458,0.087356,2.69,0,0.0,night,1.0
3,3,0.0,1.979649,-0.184949,-1.064206,0.120125,-0.215238,-0.648829,-0.087826,-0.035367,...,-0.042939,0.000799,-0.096148,-0.057780,-0.073839,1.00,0,0.0,night,1.0
4,4,0.0,1.025898,-0.171827,1.203717,1.243900,-0.636572,1.099074,-0.938651,0.569239,...,-0.262813,0.257834,-0.252829,0.108338,0.021051,1.00,0,0.0,night,1.0


In [40]:
X = train.drop(columns=["id", "Class"])
y = train.Class
X_test = test.drop(columns=["id"])

# Preprocessing

## TODO:
1. One hot every categorical column but Hour. Use LeaveOneOut for Hour.

In [41]:
numerical_feats = list(set(X.columns) - set(["TimeOfDay", "Day", "Hour"]))

In [42]:
# encoding
# feats_to_one_hot = ["TimeOfDay", "Day"]
feats_to_loo = ["Hour", "TimeOfDay", "Day"]

## leave one out
loo = LeaveOneOutEncoder()
loo.fit(X[feats_to_loo], y)
X[feats_to_loo] = loo.transform(X[feats_to_loo])
X_test[feats_to_loo] = loo.transform(X_test[feats_to_loo])

## one_hotting
# X = pd.get_dummies(X)


# scaling
feats_to_scale = numerical_feats
sc = StandardScaler()
sc.fit(X[feats_to_scale])
X[feats_to_scale] = sc.transform(X[feats_to_scale])
X_test[feats_to_scale] = sc.transform(X_test[feats_to_scale])

In [43]:
X.shape

(219129, 33)

# Modelling

In [44]:
def cross_validate(X, y, model):
    N_FOLDS = 8
    all_scores = np.zeros(N_FOLDS)

    skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=1337)

    for fold_id, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        
        X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]

               
        model.fit(X_tr, y_tr, 
                  eval_set=[(X_val, y_val)],
                  early_stopping_rounds=50,
                 verbose=False)
        
        y_pred = model.predict_proba(X_val)[:, 1]
        
        auc = roc_auc_score(y_val, y_pred)
        
        print(f"Fold {fold_id} \t auc: {auc}")
        
        all_scores[fold_id] = (auc)
    
    avg_auc = np.mean(all_scores)
    
    print(f"Avg AUC: {avg_auc}")

In [45]:
xgb_clf = xgb.XGBClassifier(tree_method="gpu_hist")

In [48]:
cross_validate(X, y, xgb_clf)

Fold 0 	 auc: 0.7885467185317059
Fold 1 	 auc: 0.8224168839737742
Fold 2 	 auc: 0.8101395054859795
Fold 3 	 auc: 0.8316166387226758
Fold 4 	 auc: 0.7715932401828614
Fold 5 	 auc: 0.8184812239704128
Fold 6 	 auc: 0.8208640396679128
Fold 7 	 auc: 0.8208553579711619
Avg AUC: 0.8105642010633105


# Tuning XGBoost 

In [50]:
def objective_xgb(trial, X, y):
    params = {
        'tree_method': "gpu_hist",
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 2, 20),
        'learning_rate': trial.suggest_loguniform('learning_rate', 0.01, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_loguniform('gamma', 0.00001, 10),
        'subsample': trial.suggest_float('subsample', 0.2, 1.0, step=0.05),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.2, 1.0, step=0.05),
        'early_stopping_rounds': trial.suggest_int("early_stoppig_rounds", 40, 100)
    }
    # we're gonna train on the combined dataset but, we'll only calculate the validation score only on comp data

    N_FOLDS = 5
    all_scores = np.zeros(N_FOLDS)

    skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=1337)

    for fold_id, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        
        X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]

        
        model = xgb.XGBClassifier(**params)
        model.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], verbose=False)
        
        y_pred = model.predict_proba(X_val)[:, 1]
                
        auc = roc_auc_score(y_val, y_pred)        
        all_scores[fold_id] = auc
    
    avg_auc = np.mean(all_scores)
    
    print(f"Avg AUC: {avg_auc}")
    
    return avg_auc

In [52]:
study_xgb = optuna.create_study(study_name="xgboost_tuning", direction="maximize")
func = lambda trial: objective_xgb(trial, X, y)
study_xgb.optimize(func, n_trials=100)

[I 2023-01-30 16:00:18,798] A new study created in memory with name: xgboost_tuning
[I 2023-01-30 16:00:24,519] Trial 0 finished with value: 0.8212660962783176 and parameters: {'n_estimators': 570, 'max_depth': 5, 'learning_rate': 0.04993716478530016, 'min_child_weight': 10, 'gamma': 0.0019182532560568532, 'subsample': 0.30000000000000004, 'colsample_bytree': 0.9000000000000001, 'early_stoppig_rounds': 52}. Best is trial 0 with value: 0.8212660962783176.


Avg AUC: 0.8212660962783176


[I 2023-01-30 16:00:29,800] Trial 1 finished with value: 0.8142354854110987 and parameters: {'n_estimators': 460, 'max_depth': 8, 'learning_rate': 0.0843632780739971, 'min_child_weight': 2, 'gamma': 4.060401743495176e-05, 'subsample': 0.2, 'colsample_bytree': 0.45, 'early_stoppig_rounds': 59}. Best is trial 0 with value: 0.8212660962783176.


Avg AUC: 0.8142354854110987


[I 2023-01-30 16:00:32,319] Trial 2 finished with value: 0.7916385563988122 and parameters: {'n_estimators': 698, 'max_depth': 3, 'learning_rate': 0.35205335830670687, 'min_child_weight': 3, 'gamma': 0.0012584641321475962, 'subsample': 0.30000000000000004, 'colsample_bytree': 0.30000000000000004, 'early_stoppig_rounds': 47}. Best is trial 0 with value: 0.8212660962783176.


...

Avg AUC: 0.813600561440523


[I 2023-01-30 16:13:25,106] Trial 98 finished with value: 0.8240176370736985 and parameters: {'n_estimators': 590, 'max_depth': 14, 'learning_rate': 0.026059708223839476, 'min_child_weight': 4, 'gamma': 0.00020270352014700206, 'subsample': 0.55, 'colsample_bytree': 0.55, 'early_stoppig_rounds': 52}. Best is trial 32 with value: 0.8310300896709879.


Avg AUC: 0.8240176370736985


[I 2023-01-30 16:13:32,104] Trial 99 finished with value: 0.8244837623738592 and parameters: {'n_estimators': 420, 'max_depth': 5, 'learning_rate': 0.038268803606100785, 'min_child_weight': 5, 'gamma': 2.38494934467478e-05, 'subsample': 0.7, 'colsample_bytree': 0.95, 'early_stoppig_rounds': 41}. Best is trial 32 with value: 0.8310300896709879.


Avg AUC: 0.8244837623738592


In [55]:
print(f"{'*'*15}XGBoost Tuning Results {'*'*15}\n\n"\
          f"BEST_VALUE: {study_xgb.best_value}\n\n"
          f"BEST_PARAMS: {study_xgb.best_params}"
     )

***************XGBoost Tuning Results ***************

BEST_VALUE: 0.8310300896709879

BEST_PARAMS: {'n_estimators': 414, 'max_depth': 5, 'learning_rate': 0.047688350515820065, 'min_child_weight': 10, 'gamma': 9.777458958828332e-05, 'subsample': 0.6000000000000001, 'colsample_bytree': 0.8500000000000001, 'early_stoppig_rounds': 52}


# Tuning LightGBM

In [83]:
from optuna.integration import LightGBMPruningCallback

def objective_lgbm(trial, X, y):
    param_grid = {
        "device_type": "gpu",
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "num_rounds": trial.suggest_int("num_rounds", 100, 500),
        "learning_rate": trial.suggest_float("learning_rate", 0.0001, 1.0),
        "num_leaves": trial.suggest_int("num_leaves", 10, 200),
        "max_depth": trial.suggest_int("max_depth", 2, 20),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 10, 200),
        "lambda_l1": trial.suggest_loguniform('lambda_l1', 0.01, 10),
        "lambda_l2": trial.suggest_loguniform('lambda_l2', 0.01, 10),
        "min_gain_to_split": trial.suggest_float("min_gain_to_split", 0.01, 15),
        "bagging_fraction":  trial.suggest_loguniform('bagging_fraction', 0.2, 1.0),
        "feature_fraction": trial.suggest_loguniform('feature_fraction', 0.2, 1.0),
        "early_stopping_rounds": trial.suggest_int("early_stopping_rounds", 50, 200)
    }

    
    
    N_FOLDS = 8
    all_scores = np.zeros(N_FOLDS)

    skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=1337)

    for fold_id, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        
        X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]
            
        model = lgbm.LGBMClassifier(objective="binary", is_unbalance=True, **param_grid)
        model.fit(
            X_tr,
            y_tr,
            eval_set=[(X_val, y_val)],
            eval_metric="auc",
            verbose=-1,
        )
        y_preds = model.predict_proba(X_val)[:, 1]
        all_scores[fold_id] = roc_auc_score(y_val, y_preds)
    
    auc = np.mean(all_scores)
    print(f"AVG CV AUC: \t {auc}")
    return auc

In [84]:
study_lgbm = optuna.create_study(direction="maximize", study_name="LGBM Tuning")
func = lambda trial: objective_lgbm(trial, X, y)
study_lgbm.optimize(func, n_trials=100, show_progress_bar=True)

[I 2023-01-30 17:52:26,484] A new study created in memory with name: LGBM Tuning


  0%|          | 0/100 [00:00<?, ?it/s]

[LightGBM] [Warning] lambda_l2 is set=0.08959300755108116, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.08959300755108116
[LightGBM] [Warning] early_stopping_round is set=129, early_stopping_rounds=129 will be ignored. Current value: early_stopping_round=129
[LightGBM] [Warning] bagging_fraction is set=0.39584607013493944, subsample=1.0 will be ignored. Current value: bagging_fraction=0.39584607013493944
[LightGBM] [Warning] feature_fraction is set=0.8306731605955961, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8306731605955961
[LightGBM] [Warning] min_data_in_leaf is set=60, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=60
[LightGBM] [Warning] num_iterations is set=462, num_rounds=462 will be ignored. Current value: num_iterations=462
[LightGBM] [Warning] min_gain_to_split is set=7.691706918724669, min_split_gain=0.0 will be ignored. Current value: min_gain_to_split=7.691706918724669
[LightGBM] [Warning] lambda_l1 is se

[LightGBM] [Fatal] Check failed: (best_split_info.left_count) > (0) at /usr/local/src/LightGBM/src/treelearner/serial_tree_learner.cpp, line 653 .



[W 2023-01-30 18:15:36,433] Trial 43 failed with parameters: {'n_estimators': 553, 'num_rounds': 148, 'learning_rate': 0.1516010227088807, 'num_leaves': 11, 'max_depth': 16, 'min_data_in_leaf': 26, 'lambda_l1': 0.4058699499893747, 'lambda_l2': 0.019698186826778995, 'min_gain_to_split': 6.617758481098144, 'bagging_fraction': 0.6134409783987361, 'feature_fraction': 0.46372469263998767, 'early_stopping_rounds': 184} because of the following error: LightGBMError('Check failed: (best_split_info.left_count) > (0) at /usr/local/src/LightGBM/src/treelearner/serial_tree_learner.cpp, line 653 .\n').
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_23/324840933.py", line 2, in <lambda>
    func = lambda trial: objective_lgbm(trial, X, y)
  File "/tmp/ipykernel_23/544366158.py", line 38, in objective_lgbm
    verbose=-1,
  File "/opt/conda/lib/python3.7/site

LightGBMError: Check failed: (best_split_info.left_count) > (0) at /usr/local/src/LightGBM/src/treelearner/serial_tree_learner.cpp, line 653 .


### best for now!
{'n_estimators': 625, 'num_rounds': 128, 'learning_rate': 0.0016209454169685287, 'num_leaves': 167, 'max_depth': 3, 'min_data_in_leaf': 106, 'lambda_l1': 0.12582407806544235, 'lambda_l2': 0.03324581293013046, 'min_gain_to_split': 10.560189522017625, 'bagging_fraction': 0.9946615300185752, 'feature_fraction': 0.4012736229918392, 'early_stopping_rounds': 140}.

### BEST v2 - again error stopped the optuna midway
Trial 40 finished with value: 0.820944282533944 and parameters: {'n_estimators': 536, 'num_rounds': 180, 'learning_rate': 0.000500503296430395, 'num_leaves': 15, 'max_depth': 14, 'min_data_in_leaf': 31, 'lambda_l1': 0.47264909156935453, 'lambda_l2': 0.024617360718187438, 'min_gain_to_split': 7.212881407911029, 'bagging_fraction': 0.6464435714593756, 'feature_fraction': 0.4710539166703898, 'early_stopping_rounds': 182}. Best is trial 40 with value: 0.820944282533944.

# Tuning CatBoost

In [80]:
def objective_cat(trial, X, y):
    param = {
        "task_type": "GPU"
        "n_estimators": trial.suggest_int("n_estimators", 100, 2000),
        "loss_function": trial.suggest_categorical("loss_function", ["CrossEntropy"]),
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-5, 1e0),
        "l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 1e-2, 1e0),
        "colsample_bylevel": trial.suggest_loguniform("colsample_bylevel", 0.01, 1.0),
        "depth": trial.suggest_int("depth", 1, 10),
        "boosting_type": trial.suggest_categorical("boosting_type", ["Ordered", "Plain"]),
        "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 2, 20),
        "one_hot_max_size": trial.suggest_int("one_hot_max_size", 2, 20),
        "early_stopping_rounds": trial.suggest_int("early_stopping_rounds", 50, 200)
    }
    # Conditional Hyper-Parameters
    if param["bootstrap_type"] == "Bayesian":
        param["bagging_temperature"] = trial.suggest_float("bagging_temperature", 0, 10)
    elif param["bootstrap_type"] == "Bernoulli":
        param["subsample"] = trial.suggest_float("subsample", 0.1, 1)
    
    N_FOLDS = 8
    all_scores = np.zeros(N_FOLDS)

    skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=1337)

    for fold_id, (train_idx, val_idx) in enumerate(skf.split(X, y)):
        
        X_tr, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_tr, y_val = y.iloc[train_idx], y.iloc[val_idx]

        cat_model = catboost.CatBoostClassifier(**param)
        cat_model.fit(X_tr, y_tr, eval_set=[(X_val, y_val)], verbose=False)
        
        y_preds = cat_model.predict_proba(X_val)[:, 1]
        all_scores[fold_id] = roc_auc_score(y_val, y_preds)
    
    auc = np.mean(all_scores)
    print(f"AVG CV AUC: \t {auc}")
    return auc

In [ ]:
study_cat = optuna.create_study(direction="maximize", study_name="CatBoost Tuning")
func = lambda trial: objective_cat(trial, X, y)
study_cat.optimize(func, n_trials=100, show_progress_bar=True)